## Location Data Cleaning

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
DATA_DIR = '../../../data/Data_Science_Listings'
df = pd.read_csv(f'{DATA_DIR}/Data_Scientist_Job_Listings_Data.csv')

In [49]:
#Before
df.original_work_type.value_counts()

On-site    3258
Remote     2999
Hybrid     1479
Name: original_work_type, dtype: int64

In [50]:
#Useful dict
us_state_to_abbrev = {
    "Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA","Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA","Hawaii": "HI","Idaho": "ID","Illinois": "IL",
    "Indiana": "IN","Iowa": "IA","Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD","Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO","Montana": "MT",
    "Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ","New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH","Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA",
    "Rhode Island": "RI","South Carolina": "SC","South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT","Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY",
    "District of Columbia": "DC","American Samoa": "AS","Guam": "GU","Northern Mariana Islands": "MP","Puerto Rico": "PR","United States Minor Outlying Islands": "UM","U.S. Virgin Islands": "VI"
}

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

india_state_dict = {"Bengaluru": "Karnataka", "Mumbai": "Maharashtra", "Delhi": "Delhi", "Kolkata": "West Bengal"} 

In [51]:
#Drop zipcode column as it has no values
df.drop(['original_location_zipcode'],axis=1,inplace=True)

#Rename columns
df.rename({'original_location_full':'Location','original_location_city':'City','original_location_state':'State','original_location_country':'Country'}, axis='columns',inplace=True)

#Clean state labels
df.State = df.State.str[0:2]

#Correcting capitalization on countries
df.Country = df.Country.str.replace('INDIA','India')
df.Country = df.Country.str.replace('USA','United States')

#Fill in remote & hybrid work type where present in location data
df['temp_remote'] = np.where((df.Location.str.contains('Remote')) & (df.Location.notnull()), 'Remote', 'other')
df['temp_hybrid'] = np.where((df.Location.str.contains('Hybrid')) & (df.Location.notnull()), 'Hybrid', 'other')
df.temp_remote = df.temp_remote.replace('other', np.nan)
df.temp_hybrid = df.temp_hybrid.replace('other', np.nan)
df.original_work_type.fillna(df.temp_hybrid,inplace=True)
df.original_work_type.fillna(df.temp_remote,inplace=True)
df.drop(['temp_remote'],axis=1,inplace=True)
df.drop(['temp_hybrid'],axis=1,inplace=True)

#Clean location data from references to remote or hybrid
df.Location = df.Location.str.strip('+123456789 ')
df.Location = df.Location.str.replace('Hybrid remote in','')
df.Location = df.Location.str.replace('Hybrid remote ','')
df.Location = df.Location.str.replace('Hybrid remote','')
df.Location = df.Location.str.replace('Remote in ','')
df.Location = df.Location.str.replace('Remote ','')
df.Location = df.Location.str.replace('locations','')
df.Location = df.Location.str.replace('location','')
df.Location = df.Location.str.replace('Temporarily ','')
df.Location = df.Location.replace('Remote', np.nan)

#Split location data into categories to deal with each part separately
df[['value1','value2','value3']] = df.Location.str.split(',', expand = True)
df.value2 = df.value2.str.strip()
df.value3 = df.value3.str.strip()

# Drop third value as it just duplicates existing country data
df.drop(['value3'],axis=1,inplace=True)

#Clean second value and transfer to state
df.loc[df.Country == 'United States','value2'] = df.value2.str[0:2]
df.value2 = df.value2.replace('Lo', np.nan)
df.State.fillna(df.value2, inplace=True)
df.drop(['value2'],axis=1,inplace=True)
df.State = df.State.apply(lambda x: abbrev_to_us_state[x] if x in abbrev_to_us_state else x)

#Clean first value and transfer to City
df.value1 = df.value1.str.strip('+123456789 ')
df.City.fillna(df.value1, inplace=True)
df.drop(['value1'],axis=1,inplace=True)
df.City = df.City.replace('India', np.nan)
df.City = df.City.replace('United States', np.nan)
df.City = df.City.replace('', np.nan)
df.City = df.City.str.replace(' metropolitan area','')
df.City = df.City.str.replace(' Metropolitan Region','')
df.City = df.City.str.replace('Greater ','')
df.City = df.City.str.replace(' Bay Area','')
df.City = df.City.str.replace(' Area','')
df.City = df.City.str.replace(' Urban district','')
df.City = df.City.str.replace(' district','')
df.State.fillna(df.City.apply(lambda x: india_state_dict[x] if x in india_state_dict else np.nan), inplace=True)
df.State.fillna(df.City.apply(lambda x: x if x in us_state_to_abbrev else np.nan), inplace=True)
df.City = df.City.apply(lambda x: np.nan if x in us_state_to_abbrev else x)

#Remove original location data
df.drop(['Location'],axis=1,inplace=True)

In [52]:
#Before
df.original_work_type.value_counts()

Remote     3610
On-site    3258
Hybrid     1554
Name: original_work_type, dtype: int64

In [53]:
df.head()

,source_id,original_record_id,original_job_posting_site,original_job_title,cleansed_job_title,original_company_name,original_industry,City,State,Country,original_salary,cleansed_salary_min,cleansed_salary_max,cleansed_salary_avg,original_job_type,original_work_type,original_job_description1,original_job_description2,posting_date
0,1,0,INDEED,Data Scientist,data scientist,Driven Brands,NaN,Benicia,California,United States,NaN,NaN,NaN,NaN,Undefined,NaN,You’ll be working alongside a team of eight an...,We invite you to join us at Driven Brands!\r\n...,11/20/2022
1,1,1,INDEED,Business Analyst,analyst,Sabot Consulting,NaN,NaN,NaN,United States,$80 - $120 an hour,NaN,NaN,NaN,Undefined,Remote,Preferred candidates will have prior experienc...,Sabot Consulting (Sabot) is a management consu...,11/20/2022
2,1,2,INDEED,IT Business Intelligence Developer (FT) Remote...,data analytics,Ballad Health,NaN,Blountville,Tennessee,United States,NaN,NaN,NaN,NaN,Undefined,Remote,Job Details Apply Save Print this job Email a…,Job Details\r\nApply\r\nSave\r\nPrint this job...,11/20/2022
3,1,3,INDEED,Data Engineer,data engineer,Longevity Holdings Inc.,NaN,Minneapolis-Saint Paul,Minnesota,United States,"$90,000 - $110,000 a year",90000.0,110000.0,100000.0,Undefined,Remote,Incorporate core data management competencies ...,Position: Data Engineer\r\nLocation: MN\r\nAs ...,11/20/2022
4,1,4,INDEED,Network Administrator/dba developer,na,WKI Kenworth,NaN,Wichita,Kansas,United States,"$50,000 - $70,000 a year",50000.0,70000.0,60000.0,Undefined,NaN,The Network Administrator provides 2nd level e...,Full Job Description\r\nThe Network Administra...,11/20/2022


In [55]:
df.to_csv(f'{DATA_DIR}/Data_Scientist_Job_Listings_Data.csv')